### ETL

Comenzamos el proyecto descargando las librerias necesarias

In [2]:
import numpy as np  
import pandas as pd

In [ ]:
data = pd.read_csv("movies_dataset.csv")

Realizamos una breve exploracion del dataset. 

In [ ]:
data.info()

Para ver el tipos de datos y cantidad de nulos por columna,ademas vemos los nulos totales del dataset

In [ ]:
data.isna().sum()

Vemos los primeros 5 registros.

In [ ]:
data.head()


Notamos que en algunos de sus campos como belongs to collection,genres,spoken_languages,production companies los valores son de tipo diccionarios y son listas de diccionarios por lo que vamos a generar funciones para posteriormente desaplanar los datos.

In [ ]:
data["belongs_to_collection"][0]

In [ ]:
type(data["belongs_to_collection"][0])

Al ser un string el tipo de dato utilizaremos eval.

In [ ]:
help(eval)

In [2]:
#La funcion recibe como parametro un string con un diccionario adentro y retorna un string con el valor de la clave "name" en caso de ser posible y en caso de que no retorna un valor NaN
def obtener_valor(diccionario):
    try:
        elemento = eval(diccionario)
        valor = elemento["name"]
        return valor
    except:
        return np.NaN
    

In [128]:
#La funcion recibe como parametro una lista de diccionarios y retorna una lista con los valores de la clave "name" de cada diccionario
def convertir_a_lista(lista_diccionario):
    lista_retornar = []
    if eval(lista_diccionario):
        lista = (eval(lista_diccionario))
        for elemento in lista:
            try:
                lista_retornar.append(elemento["name"])
            except:
                pass
    return lista_retornar
    

Buscamos registros duplicados.

Explorando los datos vimos que las peliculas tienen un id unico,pero hay 30 registros duplicados por la columna id,pudiendo haber peliculas distintas que compartan id exploramos esos registros y notamos que de los cuales 13 son perfectamente registros duplicados y los otros tienen valores inconsistentes en la columna popularity tanto por su tipo dato como pors valores disntitos.Pense en la posibilidad de que sean relanzamientos pero queda descartado a tener la misma fecha en release_date.


In [ ]:
data.duplicated().sum()

In [ ]:
data["id"].duplicated().sum()


In [ ]:
data[data["id"] == "132641"]

In [ ]:
print("Valores diferentes para pelicula Tsuma" ,data.loc[838]["popularity"],"≠",data.loc[30001]["popularity"])


Los registros duplicados los eliminamos para no tener registros redundantes,en este caso aunque sean pocos aun puede sesgar los resultados de nuestros analisis

In [ ]:
data.drop_duplicates(subset= ["id"], inplace = True, keep = "first")

Verificamos la calidad de los datos en la columna id

In [1]:
for i,elemento in enumerate(data["id"]):
    if elemento.isdigit() == False:
        print(i,elemento)

NameError: name 'data' is not defined

Vimos esos registros y notamos que toda su informacion es erronea por lo que decidimos eliminarlos

In [ ]:
data.iloc[19722]

In [ ]:
data.iloc[[19722,29483,35564]]

In [ ]:
data.drop(data.index[[19722,29483,35564]], inplace= True)

Al eliminar registros las etiquetas de los registros y su indice  difieren por lo que reseteo el indice ya que trabajare con estos para localizar registros por etiqueta gracias a loc.

In [ ]:
data.reset_index(drop=True,inplace=True)

Para seguir con el proceso debemos agregar datos.Cargamos un csv donde hay datos sobre el staff de las peliculas y sus id,vamos a separar los nombres de actores y directores que es los datos que nos interesan y luego haremos un merge.

In [124]:
data_staff = pd.read_csv("credits.csv")

Vemos los primeros 5 registros

In [125]:
data_staff.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Desaplanamos y creamos columnas con los datos que necesitamos.

In [126]:
data_staff["cast"][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

Vemos que es una lista de diccionarios donde cada diccionario representa una persona del equipo.Vamos a crear una columna con los nombres de los acotres

In [129]:
data_staff["nombres_actores"] = data_staff["cast"].apply(convertir_a_lista)

In [131]:
data_staff["nombres_actores"][0]

['Tom Hanks',
 'Tim Allen',
 'Don Rickles',
 'Jim Varney',
 'Wallace Shawn',
 'John Ratzenberger',
 'Annie Potts',
 'John Morris',
 'Erik von Detten',
 'Laurie Metcalf',
 'R. Lee Ermey',
 'Sarah Freeman',
 'Penn Jillette']

In [ ]:
data_staff["nombres_actores"][10010]

eliminamos la columna cast

In [ ]:
data_staff.drop( columns = ["cast"], inplace=True)

Ahora desaplanamos la columna crew para buscar a los directores

In [ ]:
def convertir_a_lista_director(lista_diccionario):
    lista_retornar = []
    if eval(lista_diccionario):
        lista = (eval(lista_diccionario))
        for elemento in lista:
            try:
                if elemento["job"] == "Director":
                    lista_retornar.append(elemento["name"])
            except:
                pass
    return lista_retornar
    

In [ ]:
data_staff["nombres_directores"] = data_staff["crew"].apply(convertir_a_lista_director)

In [ ]:
data_staff["nombres_directores"].tail()

In [ ]:
data_staff.drop( columns = ["crew"], inplace=True)

Eliminamos los registros con id repetidos.Y verificamos que sus valores sean consistentes.

In [ ]:
data_staff["id"]

In [ ]:
data_staff.drop_duplicates(subset = ["id"], inplace = True, keep = "first")

In [ ]:
calidad_id = data_staff['id'].apply(lambda x: isinstance(x, int)).all()
calidad_id

Cambiamos el tipo de datos id a str

In [ ]:
data_staff["id"] = data_staff["id"].astype(str)

Generamos un merge,lo hacemos por izquierda con how = "left" para que nos mantenga todos los registros de movies_dataset y agrege las columnas para actores y directores desde credits donde haya una coincidencia en los id y los demas valores de las columnas se rellenan con NaN.

In [ ]:
data_completa = pd.merge(data,data_staff, how="left")

In [ ]:
data_completa.iloc[1039]

In [ ]:
data_completa.shape

In [ ]:
data_completa[data_completa["nombres_actores"].isna()]

Vemos que el registro tiene demasiados datos faltantes y tomamos la decision de eliminarlo

In [ ]:
data_completa.drop([42853],inplace = True)

Ahora con la data completa la guardamos como un archivo csv

In [ ]:
data_completa.to_csv("data_completa.csv", index = False, sep= ",")

In [3]:
data_completa = pd.read_csv("data_completa.csv")

In [5]:
data_completa.shape

(45432, 26)

In [6]:
data_completa.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'nombres_actores', 'nombres_directores'],
      dtype='object')

Eliminamos las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage

In [24]:
columnas_a_eliminar = ["adult","imdb_id","video","original_title","poster_path","homepage"]
data_completa = data_completa.drop(columnas_a_eliminar,axis = 1)

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [25]:
data_completa["revenue"].fillna(0,inplace=True)
data_completa["budget"].fillna(0,inplace=True)

Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.Para esto exploramos las columnas revenue y budget.Primero vemos la calidad de los datos en la columna budget.

In [28]:
type(data_completa["budget"][0])

numpy.int64

In [31]:
enteros = data_completa['budget'].apply(lambda x: isinstance(x,int)).all()
enteros

True

Verifico que los valores de la columna revenue sean consistentes

In [32]:
flotantes = data_completa['revenue'].apply(lambda x: isinstance(x, float)).all()
flotantes

True

Creamos la columna return

In [33]:
#creamos la columna aplicando una funcion anonima para cada registro que tomara el valor de la division si estan los datos y 0 en caso de que no lo esten
data_completa["return"] = data_completa.apply(lambda row: row["revenue"] / row["budget"] if row["budget"] != 0 else 0, axis=1)

In [34]:
data_completa[["revenue","budget","return"]]

,revenue,budget,return
0,373554033.0,30000000,12.451801
1,262797249.0,65000000,4.043035
2,0.0,0,0.000000
3,81452156.0,16000000,5.090760
4,76578911.0,0,0.000000
...,...,...,...
45427,0.0,0,0.000000
45428,0.0,0,0.000000
45429,0.0,0,0.000000
45430,0.0,0,0.000000


Los valores nulos del campo release date deben eliminarse.Para esto exploramos la columna y buscamos inconsistencia en los datos

In [35]:
for i,elemento in enumerate(data_completa["release_date"]):
    if type(elemento) != str:
        continue
    if len(elemento) != 10:
        print(i,elemento)

Vemos los valores faltantes

In [36]:
data_completa[data_completa["release_date"].isna()]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,nombres_actores,nombres_directores,return
711,NaN,0,[],365371,en,Seven New Zealand women speak about their live...,0.005625,[],[],NaN,...,95.0,[],Released,NaN,War Stories Our Mother Never Told Us,0.0,0.0,[],[],0.0
734,NaN,0,[],215107,en,Vermont is for Lovers is an independently prod...,0.750000,[],[],NaN,...,88.0,[],Released,NaN,Vermont Is for Lovers,0.0,0.0,"['George Thrush', 'Marya Cohn', ""Ann O'Brien""]","[""John O'Brien""]",0.0
3459,NaN,0,"[{'id': 18, 'name': 'Drama'}]",94214,en,"Jails, Hospitals &amp; Hip-Hop is a cinematic ...",0.009057,[],[],NaN,...,90.0,[],NaN,three worlds / two million voices / one genera...,"Jails, Hospitals & Hip-Hop",0.0,0.0,[],[],0.0
3627,NaN,0,[],207731,en,"Tommy, a talented Puerto Rican painter living ...",0.035352,[],[],NaN,...,105.0,[],Released,NaN,Boricua's Bond,2.0,1.0,[],['Val Lik'],0.0
5878,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",99885,en,Four friends fight an insane man's crusade to ...,0.001181,[],[],NaN,...,87.0,[],Released,His Judgement Cometh...,Divine Intervention,0.0,0.0,[],[],0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45115,NaN,0,[],438910,ru,Engineering Red - 1993 Dir: Andrey I. Y. Petr...,0.001586,[],[],NaN,...,76.0,[],Released,NaN,Engineering Red,6.0,2.0,[],[],0.0
45170,NaN,0,"[{'id': 9648, 'name': 'Mystery'}, {'id': 878, ...",433711,en,"In a no holds barred documentary, acclaimed jo...",0.000220,[],[],NaN,...,74.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,All Superheroes Must Die 2: The Last Superhero,4.0,1.0,"['Tallay Wickham', 'Jason Trost']",['Jason Trost'],0.0
45304,NaN,0,[],335251,en,An exploration of the musical and social origi...,0.000000,[],[],NaN,...,0.0,[],Released,NaN,The Land Where the Blues Began,0.0,0.0,[],[],0.0
45376,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",449131,ru,NaN,0.008903,[],"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",NaN,...,NaN,[],Released,NaN,Aprel,6.0,1.0,"['Evgeniy Stychkin', 'Denis Burgazliev', 'Alek...",['Konstantin Murzenko'],0.0


Eliminamos los registros.

In [37]:
data_completa.dropna(subset="release_date",inplace=True)

In [38]:
data_completa.reset_index(drop=True,inplace=True)

De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [39]:
data_completa["release_date"] = pd.to_datetime(data_completa["release_date"],format="%Y-%m-%d",errors="coerce")

Aunque no lo pide ya que los la informacion sera solicitada en base al dia y el mes creo columnas para esto

In [40]:
lista_month = []
for elemento in data_completa["release_date"]:
    lista_month.append((elemento.month))
data_completa["release_mont"] = lista_month

In [41]:
data_completa["release_day"] = data_completa["release_date"].apply(lambda x : x.day_name())

In [42]:
data_completa["release_year"] = data_completa["release_date"].apply(lambda x : x.year)


Utilizamos las funciones creadas para desaplanar algunas columnas.Comenzamos con las colecciones
En el analisis exploratorio notamos que habia valores erroneos de tipo numerico pero la funcion lo resolvera retornando valores NaN

In [43]:
data_completa["collection"] = data_completa["belongs_to_collection"].apply(obtener_valor)
data_completa["collection"]

0                  Toy Story Collection
1                                   NaN
2             Grumpy Old Men Collection
3                                   NaN
4        Father of the Bride Collection
                      ...              
45340                               NaN
45341                               NaN
45342                               NaN
45343                               NaN
45344                               NaN
Name: collection, Length: 45345, dtype: object

Convertimos a lista la columna production companies

In [44]:
data_completa["productora"] = data_completa["production_companies"].apply(convertir_a_lista)
data_completa["productora"]

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
                               ...                        
45340    [Westdeutscher Rundfunk (WDR), Working Title F...
45341                                        [Sine Olivia]
45342                            [American World Pictures]
45343                                          [Yermoliev]
45344                                                   []
Name: productora, Length: 45345, dtype: object

In [45]:
data_completa.iloc[15358]

belongs_to_collection    {'id': 463068, 'name': 'Täällä Pohjantähden al...
budget                                                             6153120
genres                   [{'id': 18, 'name': 'Drama'}, {'id': 36, 'name...
id                                                                   41142
original_language                                                       en
overview                 Perustuu Väinö Linnan samannimiseen romaanitri...
popularity                                                        0.320384
production_companies                                                    []
production_countries             [{'iso_3166_1': 'FI', 'name': 'Finland'}]
release_date                                           2009-12-04 00:00:00
revenue                                                                0.0
runtime                                                              193.0
spoken_languages                    [{'iso_639_1': 'fi', 'name': 'suomi'}]
status                   

Creamos una funcion que recibe como parametro un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.

In [47]:
def  cantidad_filmaciones_mes(mes):
    mes = mes.capitalize()
    meses = ["Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre"]
    for i,elemento in enumerate(meses):
        if mes == elemento:
            mont = i+1
            break
    cantidad = (data_completa[data_completa["release_mont"] == mont]).shape[0]
    return f"{cantidad},cantidad de películas que fueron estrenadas en el mes de {mes}"

In [48]:
cantidad_filmaciones_mes("enero")

'5909,cantidad de películas que fueron estrenadas en el mes de Enero'

Creamos una funcion que recibe como parametro un dia en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el dia consultado en la totalidad del dataset.

In [50]:
def cantidad_filmaciones_dia(dia):
   dia = dia.capitalize()
   if dia == "Lunes":
      day = "Monday"
   elif dia == "Martes":
      day = "Tuesday"
   elif dia == "Miercoles":
      day = "Wednesday"
   elif dia == "Jueves":
      day = "Thursday"
   elif dia == "Viernes":
      day = "Friday"
   elif dia == "Sabado":
      day = "Saturday"
   else:
      day = "Sunday"
   cantidad = (data_completa[data_completa["release_day"] == day]).shape[0]
   return f"{cantidad},cantidad de películas que fueron estrenadas en los dias {dia}"

In [51]:
cantidad_filmaciones_dia("miercoles")

'7027,cantidad de películas que fueron estrenadas en los dias Miercoles'

Creamos una funcion que recibe como parametro un titulo de filmacion. Debe devolver el titutlo , el año del estreno y el score.Si varias peliculas tienen el mismo nombre retornamos la mas actual

In [197]:
def score_titulo( titulo_de_la_filmación ):
   try:
      dataframe = (data_completa[data_completa["title"] == titulo_de_la_filmación])
      dataframe = dataframe.sort_values("release_year", ascending=False)
      ano = dataframe["release_year"].iloc[0]
      popularidad = dataframe["popularity"].iloc[0]
      return f"La pelicula {titulo_de_la_filmación}, fue estrenada en el año {ano} con una popularidad de {popularidad}"
   except:
      return f"Lamentamos no contar con informacion sobre la pelicula {titulo_de_la_filmación}"

In [241]:
score_titulo("Jumanji")

{'titulo': 'Jumanji', 'anio': 1995, 'popularidad': 17.015539}

Creamos una funcion que recibe como parametro un titulo de filmacion. Debe devolver el titutlo ,la cantidad de votos y el valor promedio de votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.Si hay varias peliculas con el mismo nombre retornamos los valores de la mas actual

In [ ]:
def votos_titulo( titulo:str):
   '''Ingrese un nombre de pelicula y podras ver la cantidad de valoraciones que tienen y el puntaje medio de las mismas'''
   dataframe = data[data["title"] ==  titulo]
   if dataframe.shape[0] == 0:
      return {"respuesta":f"Hemos intentado encontrar informacion sobre la pelicula {titulo},pero no lo hemos logrado.Le sugerimos pruebe con esta pelicula, Jumanji"}
   if dataframe.shape[0] > 1:
      dataframe = dataframe.sort_values("release_year", ascending=False)
   anio = dataframe["release_year"].iloc[0]
   valoraciones = dataframe["vote_count"].iloc[0]
   promedio = dataframe["vote_average"].iloc[0]
   if valoraciones>=2000:
    return {'titulo': titulo, 'anio':anio , 'voto_total':valoraciones, 'voto_promedio':promedio}
   else:
    return {"respuesta": f"La película {titulo} fue estrenada en el año {anio}. La misma cuenta con un total de {valoraciones} valoraciones, con un promedio de {promedio}"}

In [230]:
votos_titulo("Jumanji")

{'titulo': 'Jumanji', 'anio': 1995, 'voto_total': 2413.0, 'voto_promedio': 6.9}

Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno.El retono nos indica cuanto dinero ha logrado vender la pelicula en proporcion a su inversion y si la media de estas es menor a 1 el actor ha generado en terminos totales,perdidas.

Como la definición no deberá considerar directores hacemos un funcion que elimine los directores del grupo de actores.Primero mostramos el caso de Tom Hanks un actor reconocido que tiene 3 peliculas como director y actor a la vez.

In [205]:
data_completa[data_completa['director_names'].apply(lambda x: "Tom Hanks"  in x)].shape

(3, 27)

In [156]:
def eliminar_directores(row):
    nombres_directores = eval(row['nombres_directores'])
    nombres_actores = eval(row['nombres_actores'])
    # Verificar si las columnas no están vacías y tienen formato válido
    if nombres_directores and nombres_actores:
        nuevos_actores = []
        for actor in nombres_actores:
            if actor not in nombres_directores:
                nuevos_actores.append(actor)
        row['nombres_actores'] = nuevos_actores
    return row

In [157]:
data_completa = data_completa.apply(eliminar_directores, axis=1)

In [210]:
def get_actor(nombre_actor):
    #Utilizamos el metodo explode para ampliar los registros,uno para cada actor de la lista y luego una funcion lambda para crear una mascara para registros donde aparesca el nombre del actor
    dataframe = data_completa[data_completa['actor_names'].apply(lambda x: nombre_actor in x)]
    cantidad = dataframe.shape[0]
    #Utilizamos una mascara para que las peliculas que tienen retorno 0 no generen ruido en el promedio
    dataframe = dataframe[dataframe["return"] != 0 ]
    retorno = round((dataframe["return"].sum()),2)
    if retorno == 0:
        return f"El actor {nombre_actor} ha participado de {cantidad} filmaciones, pero no tenemos los datos necesarios para calcular su exito"
    promedio = round((retorno/cantidad),2)
    return f"El actor {nombre_actor} ha participado de {cantidad} filmaciones, el mismo ha conseguido un retorno de {retorno} con un promedio de {promedio} por filmación"

In [212]:
get_actor("Tom Hanks")

'El actor Tom Hanks ha participado de 68 filmaciones, el mismo ha conseguido un retorno de 177.64 con un promedio de 2.61 por filmación'

def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.Como el objetivo es conocer los rendimientos de las peliculas no tomaremos en cuenta los registros que no tienen retorno.

In [93]:
def get_director(nombre_director):
       dataframe = data[data['director_names'].apply(lambda x: nombre_director in x)]
       if dataframe.shape[0] == 0:
             return f"Hemos intentado encontrar informacion sobre las peliculas dirigidas por {nombre_director},pero no lo hemos logrado.Le sugerimos pruebe con este nombre John Lasseter"
       dataframe= dataframe[dataframe["return"] != 0 ]
       retorno = round((dataframe["return"].sum()),2)
       nombre = dataframe["title"]
       fecha = dataframe["release_date"]
       retorno_ind = dataframe["return"]
       costo = dataframe["budget"]
       ganancia = dataframe["revenue"]
       diccionario = {"nombres" : nombre.to_list(),
                   "fecha_de_lanzamiento" : fecha.to_list(),
                   "retorno_de_la_pelicula" : retorno_ind.to_list(),
                    "ganancia_bruta_de_la pelicula" : ganancia.to_list(),
                    "inversion" : costo.to_list()
                    }
       return f"El retorno total generado por la direccion de {nombre_director} es de {retorno} y fue logrado gracias a las siguientes peliculas,te brindamos algunos datos para que conozcas mas de ellas {diccionario}"   


In [208]:
get_director("John Lasseter")

{'director': 'John Lasseter',
 'retorno_total_director': 27.65,
 'peliculas': ['Toy Story', "A Bug's Life", 'Toy Story 2', 'Cars', 'Cars 2'],
 'anio': ['1995-10-30',
  '1998-11-25',
  '1999-10-30',
  '2006-06-08',
  '2011-06-11'],
 'retorno_pelicula': [12.4518011,
  3.027157158333333,
  5.526298544444445,
  3.849859575,
  2.79926198],
 'budget_pelicula': [30000000, 120000000, 90000000, 120000000, 200000000],
 'revenue_pelicula': [373554033.0,
  363258859.0,
  497366869.0,
  461983149.0,
  559852396.0]}

Reescribimos el csv con los cambios para que pueda ser consumido desde el archivo.py y los endpoints de la API.Ademas solo dejaremos las columnas necesarias para los endpoinsts ya que es mas optimo cargar un dataframe mas pequeño.

In [4]:
import time

In [177]:
data_completa.rename(columns = {"release_mont":'release_month',
                                "nombres_actores":'actor_names' ,
                                "nombres_directores":'director_names'},inplace = True)

In [180]:
start_time_total = time.time()
data_completa_time = pd.read_csv("data_completa.csv")
total_time_total = time.time() - start_time_total
print("Llevo {} segundos en total.".format(total_time_total))

Llevo 1.046231985092163 segundos en total.


In [178]:
data = data_completa[["release_day","release_year","release_month","title","popularity","vote_count","vote_average","return","budget","revenue","actor_names","director_names","release_date"]]
data.to_csv("data.csv" ,index = False ,sep = ",")

In [179]:
start_time_total = time.time()
data_time = pd.read_csv("data.csv")
total_time_total = time.time() - start_time_total
print("Llevo {} segundos en total.".format(total_time_total))

Llevo 0.5777914524078369 segundos en total.


In [260]:
data[data["vote_count"]<2000]["title"]

2                   Grumpier Old Men
3                  Waiting to Exhale
4        Father of the Bride Part II
5                               Heat
6                            Sabrina
                    ...             
45340                     Robin Hood
45341            Century of Birthing
45342                       Betrayal
45343               Satan Triumphant
45344                       Queerama
Name: title, Length: 44832, dtype: object